In [1]:
import random
import numpy as np

import torch
from torchvision import transforms
from torch.nn import ModuleList
from torch import nn
import torch.optim as optim

# custom libs
from libs.PretrainedModels import AlexNet_cc, SqueezeNet_cc, ResNet18_cc
from libs.utils import get_model_name, init_model
from libs.Dataset import dst

In [2]:
random.seed(1996)
np.random.seed(1996)

## Dataset and dataLoaders

In [4]:
# parameters for dataloaders
batch_size=32
num_workers=2
drop_last=True

Create data loader

In [5]:
dst.create_data_loader(batch_size=batch_size, num_workers=num_workers, drop_last=drop_last)

## Training

In [6]:
learning_rates = {
    'alexnet': '', 
    'squeezenet': '',   
    'resnet18': '',
    'inceptionv3': ''
}
# dataset from libs dst
num_epochs = 25
momentum = 0.99
criterion = nn.CrossEntropyLoss()
# train_from_epoch
# save_each
# resume_global_step_from

In [7]:
train_from_epoch = 0
save_each = 5
resume_global_step_from = 0

In [ ]:
alexnet = init_model(creator=AlexNet_cc(), model_name=get_model_name(model_name='AlexNet', lr=str(learning_rates['alexnet'])), feature_extract=True, use_pretrained=True)
alexnet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['alexnet'], momentum=momentum, criterion=criterion, train_from_epoch=train_from_epoch, save_each=save_each, resume_global_step_from=resume_global_step_from)

In [ ]:
squeezenet = init_model(creator=SqueezeNet_cc(), model_name=get_model_name(model_name='SqueezeNet', lr=str(learning_rates['squeezenet'])), feature_extract=True, use_pretrained=True)
squeezenet.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['squeezenet'], momentum=momentum, criterion=criterion, train_from_epoch=0, save_each=5, resume_global_step_from=0 )

In [ ]:
resnet18 = init_model(creator=ResNet18_cc(), model_name=get_model_name(model_name='ResNet18', lr=str(learning_rates['resnet18'])), feature_extract=True, use_pretrained=True)
resnet18.do_train(dataset=dst, num_epochs=num_epochs, lr=learning_rates['resnet18'], momentum=momentum, criterion=criterion, train_from_epoch=0, save_each=5, resume_global_step_from=0 )